In [1]:
# Import packages
import numpy as np
import tensorflow as tf
import keras
from keras import Model
import os, glob
import resnet_modified # modified to have zero verbosity, deperacated several printing step, lr -> learning_rate in calling adam.
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from resnet_modified import ResNet
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model
from keras import layers
from keras.layers import Dense
import pandas as pd

In [2]:
(x, y), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train, x_v, y_train, y_v = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
x_train = x_train.astype('float32') / 255.0
x_val = x_v.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train_categorical_10 = to_categorical(y_train,10)
y_val_categorical_10 = to_categorical(y_v,10)
y_test_categorical_10 = to_categorical(y_test,10)


In [5]:
adv_x_tr_pgd_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_untarget.npy")
adv_x_val_pgd_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_val_untarget.npy")

adv_x_tr_pgd_by_resnet56v1_0_target_to_ll = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_target_to_ll.npy")
adv_x_val_pgd_by_resnet56v1_0_target_to_ll = np.load("adversarial_examples/gen_by_ResNet/pgd_0.376_x_val_target_to_ll.npy")

adv_x_tr_cw_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/cwl2_x_tr_untargeted.npy")
adv_x_val_cw_by_resnet56v1_0_untarget = np.load("adversarial_examples/gen_by_ResNet/cwl2_x_v_untargeted.npy")



nb_iter = 8
norm = 2

eps = 216/255
eps_iter = 96/255
These are parameters for PGD.
L2 norm, big perturbation (~ 0.83) for pgd

typical parameter for succesful pgd attack : for L2, 1/255 - 3/255, and for L infinite (norm = np.inf), 8/255

max iteration = 100 for cwl2


In [6]:
label_dict = {(3,1) : 'ResNet20v1',
        (3,2) : 'ResNet20v2',
        (9,1): 'ResNet56v1',
        (9,2): 'ResNet56v2'}

In [ ]:
## custom turn specialist function

def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 5, 10, 15, 18 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 18:
        lr *= 0.5e-3
    elif epoch > 15:
        lr *= 1e-3
    elif epoch > 10:
        lr *= 1e-2
    elif epoch > 5:
        lr *= 1e-1
#     print('Learning rate: ', lr)
    return lr

def turn_specialist(model : Model, path : str,
        x_tr: np.ndarray | None = None,
        y_tr: np.ndarray | None = None,
        x_v: np.ndarray | None = None,
        y_v: np.ndarray | None = None,
        epochs: int = 21,
        learning_rate : float = 1e-3,
        batch_size: int = 128,
        # save_each: bool = False,
        # save_bests: int | None = None,
        verbose: int = 1,
        name : str = ''
    ):
        
        # build specialist network
        base = Model(inputs = model.inputs, outputs = model.layers[-2].output, name=f"base{name}")
        x    = keras.Input(shape=base.input_shape[1:], name=f"in{name}")
        y    = Dense(10, name=f"dense{name}")(base(x)) # 10 can be changed to len(newtarget)
        z    = layers.Softmax(name=f"softmax{name}")(y)
        specialist = Model(inputs = x, outputs = z)
        specialist.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )
        




        # callbacks
        callbacks = [ModelCheckpoint(path, monitor="val_accuracy",
                                    save_best_only=True, verbose=verbose)]
        
        callbacks += [LearningRateScheduler(lr_schedule),
                        ReduceLROnPlateau(factor=np.sqrt(0.1), patience=5, min_lr=5e-7)]

        # fit 
        hist = specialist.fit(x_tr, y_tr, batch_size=batch_size,
                        validation_data=(x_v, y_v),
                        epochs=epochs, callbacks=callbacks, verbose=verbose)



        # ---------- summary ----------
        metric = "val_accuracy"
        best = np.max(hist.history[metric])
        first = hist.history[metric][0]
        print(f"best {metric} {best:.3f} (first {first:.3f})")
        return specialist, hist


In [8]:
training_y_long=np.concatenate([y_train,y_train+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)
validating_y_long=np.concatenate([y_v,y_v+10],axis=0)
validating_y_long = keras.utils.to_categorical(validating_y_long,20)

In [9]:
# ## Structure varies, adversarial is from ResNet56v1, PGD method, untargeted , L2 norm, 216/255 epsilon perturbation

# training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_untarget],axis=0)
# validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_untarget],axis=0)
# for n in [3,9]:
#     for v in [1,2]:
#         for label in [0,1,2,3]:
#             path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_untargeted-adv_{label}.keras'
#             print(f'fit starts {label_dict[(n,v)]}_ff_untargeted-adv_{label}.keras')
#             resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
#             resnet_model.train(save_best_only=True,epochs=200)
#             print(f'fit ends. Turn into specialist on original only')
#             specialist = turn_specialist(load_model(path),
#                 f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_untargeted-adv_{label}.keras', 
#                 x_train, y_train_categorical_10, 
#                 x_val, y_val_categorical_10)



In [ ]:
## PGD method, targeted to least likely logit

training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_target_to_ll],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_target_to_ll],axis=0)

for n, v, label in [(9,2,2),(9,2,3)]:
    path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras'
    print(f'fit starts {label_dict[(n,v)]}_ff_targeted-adv_{label}.keras')

    # Train full + fine model 
    resnet_model = ResNet(
        path,
        training_x, training_y_long,
        validating_x, validating_y_long,
        n=n, version=v
    )
    resnet_model.train(save_best_only=True, epochs=200)

    print(f'fit ends. Turn into specialist on original only')

    # Load best model and train specialist on original data
    specialist = turn_specialist(
        load_model(path),
        f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras',
        x_train, y_train_categorical_10,
        x_val, y_val_categorical_10
    )

fit starts ResNet56v2_ff_targeted-adv_2.keras


In [10]:
## load pre-trained CIFAR10 ResNet models
folder = 'CIFAR10models/ResNet/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

for f, model_ in loaded_models.items():
    base = f.replace('.keras','')
    parts = base.split('_')
    structure = label_dict[(int(parts[1]), int(parts[2][-1]))]
    idx = parts[-1][-1]
    

    path = f'CIFAR10models/more_tunned/{structure}_more_specialized_{idx}.keras'
    print(f'index:{idx}\nstructure:{structure}')
    specialist = turn_specialist(
        model_,
        path,
        x_train, y_train_categorical_10,    
        x_val, y_val_categorical_10
        )

index:0
structure:ResNet20v1
Epoch 1/21
313/313 [==============================] - ETA: 0s - loss: 0.4835 - accuracy: 0.9006
Epoch 1: val_accuracy improved from -inf to 0.88040, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_0.keras
313/313 [==============================] - 16s 8ms/step - loss: 0.4835 - accuracy: 0.9006 - val_loss: 0.5035 - val_accuracy: 0.8804 - lr: 0.0010
Epoch 2/21
307/313 [============================>.] - ETA: 0s - loss: 0.2742 - accuracy: 0.9581
Epoch 2: val_accuracy did not improve from 0.88040
313/313 [==============================] - 2s 6ms/step - loss: 0.2745 - accuracy: 0.9579 - val_loss: 0.5347 - val_accuracy: 0.8680 - lr: 0.0010
Epoch 3/21
307/313 [============================>.] - ETA: 0s - loss: 0.2335 - accuracy: 0.9699
Epoch 3: val_accuracy did not improve from 0.88040
313/313 [==============================] - 2s 6ms/step - loss: 0.2344 - accuracy: 0.9696 - val_loss: 0.8107 - val_accuracy: 0.8085 - lr: 0.0010
Epoch 4/21
307/31

Epoch 5/21
306/313 [============================>.] - ETA: 0s - loss: 0.2098 - accuracy: 0.9760
Epoch 5: val_accuracy did not improve from 0.87460
313/313 [==============================] - 2s 6ms/step - loss: 0.2101 - accuracy: 0.9758 - val_loss: 0.6076 - val_accuracy: 0.8643 - lr: 0.0010
Epoch 6/21
307/313 [============================>.] - ETA: 0s - loss: 0.1981 - accuracy: 0.9800
Epoch 6: val_accuracy did not improve from 0.87460
313/313 [==============================] - 2s 6ms/step - loss: 0.1985 - accuracy: 0.9798 - val_loss: 0.6770 - val_accuracy: 0.8531 - lr: 0.0010
Epoch 7/21
307/313 [============================>.] - ETA: 0s - loss: 0.1627 - accuracy: 0.9935
Epoch 7: val_accuracy improved from 0.87460 to 0.90110, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_1.keras
313/313 [==============================] - 2s 7ms/step - loss: 0.1625 - accuracy: 0.9936 - val_loss: 0.4759 - val_accuracy: 0.9011 - lr: 1.0000e-04
Epoch 8/21
307/313 [====================

313/313 [==============================] - 2s 7ms/step - loss: 0.1491 - accuracy: 0.9985 - val_loss: 0.4709 - val_accuracy: 0.9067 - lr: 1.0000e-04
Epoch 9/21
307/313 [============================>.] - ETA: 0s - loss: 0.1443 - accuracy: 0.9993
Epoch 9: val_accuracy did not improve from 0.90670
313/313 [==============================] - 2s 6ms/step - loss: 0.1443 - accuracy: 0.9994 - val_loss: 0.4720 - val_accuracy: 0.9060 - lr: 1.0000e-04
Epoch 10/21
307/313 [============================>.] - ETA: 0s - loss: 0.1409 - accuracy: 0.9998
Epoch 10: val_accuracy improved from 0.90670 to 0.90700, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_2.keras
313/313 [==============================] - 2s 7ms/step - loss: 0.1409 - accuracy: 0.9998 - val_loss: 0.4737 - val_accuracy: 0.9070 - lr: 1.0000e-04
Epoch 11/21
307/313 [============================>.] - ETA: 0s - loss: 0.1381 - accuracy: 0.9998
Epoch 11: val_accuracy improved from 0.90700 to 0.90750, saving model to CIFAR10

307/313 [============================>.] - ETA: 0s - loss: 0.1514 - accuracy: 0.9999
Epoch 16: val_accuracy did not improve from 0.90590
313/313 [==============================] - 2s 6ms/step - loss: 0.1513 - accuracy: 0.9999 - val_loss: 0.5118 - val_accuracy: 0.9058 - lr: 1.0000e-05
Epoch 17/21
307/313 [============================>.] - ETA: 0s - loss: 0.1510 - accuracy: 1.0000
Epoch 17: val_accuracy improved from 0.90590 to 0.90600, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_4.keras
313/313 [==============================] - 2s 7ms/step - loss: 0.1510 - accuracy: 1.0000 - val_loss: 0.5123 - val_accuracy: 0.9060 - lr: 1.0000e-06
Epoch 18/21
307/313 [============================>.] - ETA: 0s - loss: 0.1510 - accuracy: 0.9999
Epoch 18: val_accuracy improved from 0.90600 to 0.90630, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_4.keras
313/313 [==============================] - 2s 7ms/step - loss: 0.1510 - accuracy: 0.9999 - val_loss: 0.

Epoch 20/21
307/313 [============================>.] - ETA: 0s - loss: 0.1350 - accuracy: 1.0000
Epoch 20: val_accuracy did not improve from 0.90970
313/313 [==============================] - 2s 6ms/step - loss: 0.1350 - accuracy: 1.0000 - val_loss: 0.4708 - val_accuracy: 0.9091 - lr: 5.0000e-07
Epoch 21/21
307/313 [============================>.] - ETA: 0s - loss: 0.1348 - accuracy: 1.0000
Epoch 21: val_accuracy did not improve from 0.90970
313/313 [==============================] - 2s 6ms/step - loss: 0.1348 - accuracy: 1.0000 - val_loss: 0.4709 - val_accuracy: 0.9095 - lr: 5.0000e-07
best val_accuracy 0.910 (first 0.872)
index:6
structure:ResNet20v1
Epoch 1/21
307/313 [============================>.] - ETA: 0s - loss: 0.4901 - accuracy: 0.8988
Epoch 1: val_accuracy improved from -inf to 0.87790, saving model to CIFAR10models/more_tunned/ResNet20v1_more_specialized_6.keras
313/313 [==============================] - 3s 8ms/step - loss: 0.4876 - accuracy: 0.8996 - val_loss: 0.5181 - va

...

Epoch 8/21
313/313 [==============================] - ETA: 0s - loss: 0.1635 - accuracy: 0.9984
Epoch 8: val_accuracy improved from 0.90210 to 0.90390, saving model to CIFAR10models/more_tunned/ResNet56v1_more_specialized_2.keras
313/313 [==============================] - 5s 17ms/step - loss: 0.1635 - accuracy: 0.9984 - val_loss: 0.5032 - val_accuracy: 0.9039 - lr: 1.0000e-04
Epoch 9/21
313/313 [==============================] - ETA: 0s - loss: 0.1585 - accuracy: 0.9994
Epoch 9: val_accuracy improved from 0.90390 to 0.90510, saving model to CIFAR10models/more_tunned/ResNet56v1_more_specialized_2.keras
313/313 [==============================] - 5s 17ms/step - loss: 0.1585 - accuracy: 0.9994 - val_loss: 0.5111 - val_accuracy: 0.9051 - lr: 1.0000e-04
Epoch 10/21
313/313 [==============================] - ETA: 0s - loss: 0.1548 - accuracy: 0.9999
Epoch 10: val_accuracy improved from 0.90510 to 0.90700, saving model to CIFAR10models/more_tunned/ResNet56v1_more_specialized_2.keras
313/313 [=

Epoch 12/21
313/313 [==============================] - ETA: 0s - loss: 0.1496 - accuracy: 0.9999
Epoch 12: val_accuracy did not improve from 0.90980
313/313 [==============================] - 5s 16ms/step - loss: 0.1496 - accuracy: 0.9999 - val_loss: 0.5073 - val_accuracy: 0.9095 - lr: 1.0000e-05
Epoch 13/21
313/313 [==============================] - ETA: 0s - loss: 0.1491 - accuracy: 1.0000
Epoch 13: val_accuracy improved from 0.90980 to 0.91000, saving model to CIFAR10models/more_tunned/ResNet56v1_more_specialized_3.keras
313/313 [==============================] - 5s 17ms/step - loss: 0.1491 - accuracy: 1.0000 - val_loss: 0.5075 - val_accuracy: 0.9100 - lr: 1.0000e-05
Epoch 14/21
313/313 [==============================] - ETA: 0s - loss: 0.1490 - accuracy: 0.9999
Epoch 14: val_accuracy did not improve from 0.91000
313/313 [==============================] - 5s 16ms/step - loss: 0.1490 - accuracy: 0.9999 - val_loss: 0.5083 - val_accuracy: 0.9100 - lr: 1.0000e-05
Epoch 15/21
313/313 [==

313/313 [==============================] - 10s 32ms/step - loss: 0.1353 - accuracy: 0.9999 - val_loss: 0.3012 - val_accuracy: 0.9477 - lr: 1.0000e-05
Epoch 17/21
311/313 [============================>.] - ETA: 0s - loss: 0.1349 - accuracy: 0.9999
Epoch 17: val_accuracy did not improve from 0.94820
313/313 [==============================] - 10s 32ms/step - loss: 0.1349 - accuracy: 0.9999 - val_loss: 0.3010 - val_accuracy: 0.9478 - lr: 1.0000e-06
Epoch 18/21
313/313 [==============================] - ETA: 0s - loss: 0.1350 - accuracy: 0.9999
Epoch 18: val_accuracy did not improve from 0.94820
313/313 [==============================] - 10s 32ms/step - loss: 0.1350 - accuracy: 0.9999 - val_loss: 0.3012 - val_accuracy: 0.9475 - lr: 1.0000e-06
Epoch 19/21
311/313 [============================>.] - ETA: 0s - loss: 0.1350 - accuracy: 0.9999
Epoch 19: val_accuracy did not improve from 0.94820
313/313 [==============================] - 10s 32ms/step - loss: 0.1350 - accuracy: 0.9999 - val_loss: 

Epoch 21/21
311/313 [============================>.] - ETA: 0s - loss: 0.1460 - accuracy: 0.9999
Epoch 21: val_accuracy did not improve from 0.92220
313/313 [==============================] - 10s 32ms/step - loss: 0.1460 - accuracy: 0.9999 - val_loss: 0.4487 - val_accuracy: 0.9209 - lr: 5.0000e-07
best val_accuracy 0.922 (first 0.843)
index:2
structure:ResNet56v2
Epoch 1/21
311/313 [============================>.] - ETA: 0s - loss: 0.5058 - accuracy: 0.9000
Epoch 1: val_accuracy improved from -inf to 0.85490, saving model to CIFAR10models/more_tunned/ResNet56v2_more_specialized_2.keras
313/313 [==============================] - 15s 36ms/step - loss: 0.5051 - accuracy: 0.9002 - val_loss: 0.5989 - val_accuracy: 0.8549 - lr: 0.0010
Epoch 2/21
311/313 [============================>.] - ETA: 0s - loss: 0.2896 - accuracy: 0.9543
Epoch 2: val_accuracy improved from 0.85490 to 0.87380, saving model to CIFAR10models/more_tunned/ResNet56v2_more_specialized_2.keras
313/313 [======================

313/313 [==============================] - 10s 33ms/step - loss: 0.2437 - accuracy: 0.9665 - val_loss: 0.5704 - val_accuracy: 0.8711 - lr: 0.0010
Epoch 4/21
311/313 [============================>.] - ETA: 0s - loss: 0.2303 - accuracy: 0.9709
Epoch 4: val_accuracy did not improve from 0.87110
313/313 [==============================] - 10s 32ms/step - loss: 0.2304 - accuracy: 0.9709 - val_loss: 0.5740 - val_accuracy: 0.8688 - lr: 0.0010
Epoch 5/21
311/313 [============================>.] - ETA: 0s - loss: 0.2206 - accuracy: 0.9742
Epoch 5: val_accuracy did not improve from 0.87110
313/313 [==============================] - 10s 32ms/step - loss: 0.2206 - accuracy: 0.9742 - val_loss: 0.5950 - val_accuracy: 0.8666 - lr: 0.0010
Epoch 6/21
311/313 [============================>.] - ETA: 0s - loss: 0.2156 - accuracy: 0.9751
Epoch 6: val_accuracy improved from 0.87110 to 0.87760, saving model to CIFAR10models/more_tunned/ResNet56v2_more_specialized_3.keras
313/313 [=============================

In [ ]:

training_x = np.concatenate([x_train,adv_x_tr_pgd_by_resnet56v1_0_target_to_ll],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_pgd_by_resnet56v1_0_target_to_ll],axis=0)

for n in [3,9]:
    for v in [1,2]:
        for label in [0,1,2,3]:
            path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras'
            print(f'fit starts {label_dict[(n,v)]}_ff_targeted-adv_{label}.keras')
            resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
            resnet_model.train(save_best_only=True,epochs=200)
            print(f'fit ends. Turn into specialist on original only')
            specialist = turn_specialist(load_model(path),
                f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_targeted-adv_{label}.keras', 
                x_train, y_train_categorical_10, 
                x_val, y_val_categorical_10)




fit starts ResNet20v1_ff_targeted-adv_0.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.903 (first 0.844)
fit starts ResNet20v1_ff_targeted-adv_1.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.904 (first 0.822)
fit starts ResNet20v1_ff_targeted-adv_2.keras
fit ends. Turn into specialist on original only
best val_accuracy 0.902 (first 0.800)
fit starts ResNet20v1_ff_targeted-adv_3.keras


In [ ]:
## Now adversarial is CWL2, max iteration = 100
training_x = np.concatenate([x_train,adv_x_tr_cw_by_resnet56v1_0_untarget],axis=0)
validating_x = np.concatenate([x_val,adv_x_val_cw_by_resnet56v1_0_untarget],axis=0)

training_y_long=np.concatenate([y_train,y_train[:10001]+10],axis=0)
training_y_long = keras.utils.to_categorical(training_y_long,20)

for n, v, label in [(9,2,1),(9,2,2),(9,2,3)]:
    path = f'adversarial_models/full_and_fine/{label_dict[(n,v)]}_ff_cw-adv_{label}.keras'
    print(f'fit starts {label_dict[(n,v)]}_ff_cw-adv_{label}.keras')
    resnet_model = ResNet(path,training_x,training_y_long,validating_x,validating_y_long,n=n,version=v)
    resnet_model.train(save_best_only=True,epochs=200)
    print(f'fit ends. Turn into specialist on original only')
    specialist = turn_specialist(load_model(path),
        f'adversarial_models/specialized_from_full_and_fine/classifying_original_from_{label_dict[(n,v)]}_ff_cw-adv_{label}.keras', 
        x_train, y_train_categorical_10, 
        x_val, y_val_categorical_10, verbose = 1)







fit starts ResNet56v2_ff_cw-adv_1.keras
Epoch 1/200
1562/1563 [============================>.] - ETA: 0s - loss: 2.7946 - acc: 0.3859
Epoch 1: val_acc improved from -inf to 0.26935, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [==============================] - 71s 33ms/step - loss: 2.7942 - acc: 0.3860 - val_loss: 2.8401 - val_acc: 0.2693 - lr: 0.0010
Epoch 2/200
1563/1563 [==============================] - ETA: 0s - loss: 2.0188 - acc: 0.4949
Epoch 2: val_acc improved from 0.26935 to 0.28085, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [==============================] - 52s 33ms/step - loss: 2.0188 - acc: 0.4949 - val_loss: 2.6210 - val_acc: 0.2808 - lr: 0.0010
Epoch 3/200
1563/1563 [==============================] - ETA: 0s - loss: 1.8000 - acc: 0.5368
Epoch 3: val_acc improved from 0.28085 to 0.34590, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [=======

Epoch 27/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0053 - acc: 0.7431
Epoch 27: val_acc did not improve from 0.59435
1563/1563 [==============================] - 52s 33ms/step - loss: 1.0053 - acc: 0.7431 - val_loss: 1.5471 - val_acc: 0.5846 - lr: 0.0010
Epoch 28/200
1563/1563 [==============================] - ETA: 0s - loss: 0.9953 - acc: 0.7447
Epoch 28: val_acc improved from 0.59435 to 0.62370, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [==============================] - 51s 33ms/step - loss: 0.9953 - acc: 0.7447 - val_loss: 1.2840 - val_acc: 0.6237 - lr: 0.0010
Epoch 29/200
1563/1563 [==============================] - ETA: 0s - loss: 0.9852 - acc: 0.7487
Epoch 29: val_acc did not improve from 0.62370
1563/1563 [==============================] - 51s 32ms/step - loss: 0.9852 - acc: 0.7487 - val_loss: 1.4203 - val_acc: 0.5986 - lr: 0.0010
Epoch 30/200
1563/1563 [==============================] - ETA: 0s - loss: 0.9

1563/1563 [==============================] - ETA: 0s - loss: 0.8742 - acc: 0.7809
Epoch 55: val_acc did not improve from 0.65025
1563/1563 [==============================] - 48s 31ms/step - loss: 0.8742 - acc: 0.7809 - val_loss: 1.4305 - val_acc: 0.6388 - lr: 0.0010
Epoch 56/200
1563/1563 [==============================] - ETA: 0s - loss: 0.8726 - acc: 0.7803
Epoch 56: val_acc did not improve from 0.65025
1563/1563 [==============================] - 48s 31ms/step - loss: 0.8726 - acc: 0.7803 - val_loss: 1.6103 - val_acc: 0.5457 - lr: 0.0010
Epoch 57/200
1563/1563 [==============================] - ETA: 0s - loss: 0.8648 - acc: 0.7836
Epoch 57: val_acc improved from 0.65025 to 0.65500, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [==============================] - 49s 31ms/step - loss: 0.8648 - acc: 0.7836 - val_loss: 1.1924 - val_acc: 0.6550 - lr: 0.0010
Epoch 58/200
1563/1563 [==============================] - ETA: 0s - loss: 0.8654 - acc: 0.

1563/1563 [==============================] - 49s 31ms/step - loss: 0.6433 - acc: 0.8608 - val_loss: 0.8572 - val_acc: 0.7816 - lr: 1.0000e-04
Epoch 83/200
1563/1563 [==============================] - ETA: 0s - loss: 0.6005 - acc: 0.8733
Epoch 83: val_acc improved from 0.78160 to 0.80310, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_1.keras
1563/1563 [==============================] - 49s 31ms/step - loss: 0.6005 - acc: 0.8733 - val_loss: 0.8049 - val_acc: 0.8031 - lr: 1.0000e-04
Epoch 84/200
1563/1563 [==============================] - ETA: 0s - loss: 0.5841 - acc: 0.8763
Epoch 84: val_acc did not improve from 0.80310
1563/1563 [==============================] - 49s 31ms/step - loss: 0.5841 - acc: 0.8763 - val_loss: 0.8134 - val_acc: 0.7987 - lr: 1.0000e-04
Epoch 85/200
1563/1563 [==============================] - ETA: 0s - loss: 0.5663 - acc: 0.8814
Epoch 85: val_acc improved from 0.80310 to 0.80690, saving model to adversarial_models/full_and_fine/ResNet56v2_

1563/1563 [==============================] - ETA: 0s - loss: 0.3688 - acc: 0.9238
Epoch 137: val_acc did not improve from 0.82485
1563/1563 [==============================] - 49s 31ms/step - loss: 0.3688 - acc: 0.9238 - val_loss: 0.7537 - val_acc: 0.8207 - lr: 1.0000e-05
Epoch 138/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3694 - acc: 0.9240
Epoch 138: val_acc did not improve from 0.82485
1563/1563 [==============================] - 49s 31ms/step - loss: 0.3694 - acc: 0.9240 - val_loss: 0.7699 - val_acc: 0.8142 - lr: 3.1623e-06
Epoch 139/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3698 - acc: 0.9243
Epoch 139: val_acc did not improve from 0.82485
1563/1563 [==============================] - 48s 31ms/step - loss: 0.3698 - acc: 0.9243 - val_loss: 0.7632 - val_acc: 0.8192 - lr: 1.0000e-05
Epoch 140/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3710 - acc: 0.9234
Epoch 140: val_acc did not improve from 0.82485
1563/1563 [=

1563/1563 [==============================] - 48s 31ms/step - loss: 0.3587 - acc: 0.9264 - val_loss: 0.7569 - val_acc: 0.8232 - lr: 1.0000e-06
Epoch 166/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3589 - acc: 0.9264
Epoch 166: val_acc did not improve from 0.82550
1563/1563 [==============================] - 48s 31ms/step - loss: 0.3589 - acc: 0.9264 - val_loss: 0.7644 - val_acc: 0.8192 - lr: 1.0000e-06
Epoch 167/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3575 - acc: 0.9272
Epoch 167: val_acc did not improve from 0.82550
1563/1563 [==============================] - 48s 31ms/step - loss: 0.3575 - acc: 0.9272 - val_loss: 0.7592 - val_acc: 0.8214 - lr: 1.0000e-06
Epoch 168/200
1563/1563 [==============================] - ETA: 0s - loss: 0.3603 - acc: 0.9252
Epoch 168: val_acc did not improve from 0.82550
1563/1563 [==============================] - 48s 31ms/step - loss: 0.3603 - acc: 0.9252 - val_loss: 0.7678 - val_acc: 0.8180 - lr: 5.0000e-07


313/313 [==============================] - 24s 78ms/step - loss: 0.1551 - accuracy: 0.9999 - val_loss: 0.4655 - val_accuracy: 0.9187 - lr: 1.0000e-06
Epoch 19/21
312/313 [============================>.] - ETA: 0s - loss: 0.1551 - accuracy: 0.9999
Epoch 19: val_accuracy did not improve from 0.91920
313/313 [==============================] - 25s 78ms/step - loss: 0.1551 - accuracy: 0.9999 - val_loss: 0.4658 - val_accuracy: 0.9186 - lr: 1.0000e-06
Epoch 20/21
312/313 [============================>.] - ETA: 0s - loss: 0.1550 - accuracy: 0.9999
Epoch 20: val_accuracy did not improve from 0.91920
313/313 [==============================] - 24s 78ms/step - loss: 0.1550 - accuracy: 0.9999 - val_loss: 0.4653 - val_accuracy: 0.9192 - lr: 5.0000e-07
Epoch 21/21
312/313 [============================>.] - ETA: 0s - loss: 0.1552 - accuracy: 0.9998
Epoch 21: val_accuracy improved from 0.91920 to 0.91930, saving model to adversarial_models/specialized_from_full_and_fine/classifying_original_from_ResNet

1563/1563 [==============================] - 50s 32ms/step - loss: 1.0577 - acc: 0.7249 - val_loss: 1.5320 - val_acc: 0.5490 - lr: 0.0010
Epoch 23/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0466 - acc: 0.7269
Epoch 23: val_acc improved from 0.54895 to 0.55630, saving model to adversarial_models/full_and_fine/ResNet56v2_ff_cw-adv_2.keras
1563/1563 [==============================] - 50s 32ms/step - loss: 1.0466 - acc: 0.7269 - val_loss: 1.6608 - val_acc: 0.5563 - lr: 0.0010
Epoch 24/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0371 - acc: 0.7302
Epoch 24: val_acc did not improve from 0.55630
1563/1563 [==============================] - 49s 31ms/step - loss: 1.0371 - acc: 0.7302 - val_loss: 1.6197 - val_acc: 0.5181 - lr: 0.0010
Epoch 25/200
1563/1563 [==============================] - ETA: 0s - loss: 1.0238 - acc: 0.7343
Epoch 25: val_acc did not improve from 0.55630
1563/1563 [==============================] - 49s 32ms/step - loss: 1.0238 - a

In [14]:
# ## load pre-trained CIFAR10 ResNet models
# folder = 'CIFAR10models/ResNet/'
# pattern = os.path.join(folder, '*.keras')
# file_list = sorted(glob.glob(pattern))
# loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

# ## specialize once
# for f, model_ in loaded_models.items():
#     base = f.replace('.keras','')
#     parts = base.split('_')
#     structure = label_dict[(int(parts[1]), int(parts[2][-1]))]
#     idx = parts[-1][-1]
    

#     path = f'CIFAR10models/more_tunned/{structure}_more_specialized_{idx}.keras'
#     specialist = turn_specialist(
#         model_,
#         path,
#         x_train, y_train_categorical_10,    
#         x_val, y_val_categorical_10
#         )
    
## once more
model= load_model('CIFAR10models/more_tunned/ResNet20v1_more_specialized_0.keras')
print('accuracy (model specialized once) :',np.sum(np.argmax(model.predict(x_test),axis=-1) == y_test.reshape(-1)))
      
path = f'CIFAR10models/more_tunned/ResNet20v1_once_more_specialized_0.keras'
specialist = turn_specialist(
    model,
    path,
    x_train, y_train_categorical_10,    
    x_val, y_val_categorical_10,
    name = '_twice'
    )
print('accuracy (model specialized twice) :',np.sum(np.argmax(specialist.predict(x_test),axis=-1) == y_test.reshape(-1)))
 

313/313 [==============================] - 1s 1ms/step
accuracy (model specialized once) : 9052
Epoch 1/21
307/313 [============================>.] - ETA: 0s - loss: 0.5133 - accuracy: 0.9043
Epoch 1: val_accuracy improved from -inf to 0.86460, saving model to CIFAR10models/more_tunned/ResNet20v1_once_more_specialized_0.keras
313/313 [==============================] - 3s 8ms/step - loss: 0.5079 - accuracy: 0.9057 - val_loss: 0.6248 - val_accuracy: 0.8646 - lr: 0.0010
Epoch 2/21
307/313 [============================>.] - ETA: 0s - loss: 0.1957 - accuracy: 0.9799
Epoch 2: val_accuracy did not improve from 0.86460
313/313 [==============================] - 2s 6ms/step - loss: 0.1957 - accuracy: 0.9799 - val_loss: 0.6334 - val_accuracy: 0.8592 - lr: 0.0010
Epoch 3/21
307/313 [============================>.] - ETA: 0s - loss: 0.1930 - accuracy: 0.9809
Epoch 3: val_accuracy did not improve from 0.86460
313/313 [==============================] - 2s 6ms/step - loss: 0.1935 - accuracy: 0.9808 -

AttributeError: 'NoneType' object has no attribute 'predict'

In [18]:
## once more
model= load_model('CIFAR10models/more_tunned/ResNet20v1_once_more_specialized_0.keras')
print('accuracy (model specialized twice) :',np.sum(np.argmax(model.predict(x_test),axis=-1) == y_test.reshape(-1)))
      
path = f'CIFAR10models/more_tunned/ResNet20v1_twice_more_specialized_0.keras'
specialist = turn_specialist(
    model,
    path,
    x_train, y_train_categorical_10,    
    x_val, y_val_categorical_10,
    name = '_third'
    )
print('accuracy (model specialized third) :',np.sum(np.argmax(specialist[0].predict(x_test),axis=-1) == y_test.reshape(-1)))
 

313/313 [==============================] - 1s 1ms/step
accuracy (model specialized once) : 8976
Epoch 1/21
307/313 [============================>.] - ETA: 0s - loss: 0.6872 - accuracy: 0.8647
Epoch 1: val_accuracy improved from -inf to 0.84430, saving model to CIFAR10models/more_tunned/ResNet20v1_twice_more_specialized_0.keras
313/313 [==============================] - 4s 7ms/step - loss: 0.6796 - accuracy: 0.8664 - val_loss: 0.7150 - val_accuracy: 0.8443 - lr: 0.0010
Epoch 2/21
307/313 [============================>.] - ETA: 0s - loss: 0.1887 - accuracy: 0.9827
Epoch 2: val_accuracy improved from 0.84430 to 0.85680, saving model to CIFAR10models/more_tunned/ResNet20v1_twice_more_specialized_0.keras
313/313 [==============================] - 2s 7ms/step - loss: 0.1887 - accuracy: 0.9827 - val_loss: 0.7129 - val_accuracy: 0.8568 - lr: 0.0010
Epoch 3/21
307/313 [============================>.] - ETA: 0s - loss: 0.1912 - accuracy: 0.9808
Epoch 3: val_accuracy did not improve from 0.85680


TypeError: 'NoneType' object is not subscriptable

In [19]:
model_base = load_model('CIFAR10models/ResNet/n_3_v1_cifar10.keras')
print('accuracy (original) :',np.sum(np.argmax(model_base.predict(x_test),axis=-1) == y_test.reshape(-1)))

model_once= load_model('CIFAR10models/more_tunned/ResNet20v1_more_specialized_0.keras')
print('accuracy (model specialized once) :',np.sum(np.argmax(model_once.predict(x_test),axis=-1) == y_test.reshape(-1)))
     
model_twice= load_model('CIFAR10models/more_tunned/ResNet20v1_once_more_specialized_0.keras')
print('accuracy (model specialized twice) :',np.sum(np.argmax(model_twice.predict(x_test),axis=-1) == y_test.reshape(-1)))

model_third = load_model('CIFAR10models/more_tunned/ResNet20v1_twice_more_specialized_0.keras')
print('accuracy (model specialized three times) :',np.sum(np.argmax(model_third.predict(x_test),axis=-1) == y_test.reshape(-1)))


313/313 [==============================] - 1s 1ms/step
accuracy (original) : 8248
313/313 [==============================] - 1s 1ms/step
accuracy (model specialized once) : 9052
313/313 [==============================] - 1s 1ms/step
accuracy (model specialized twice) : 8976
313/313 [==============================] - 1s 1ms/step
accuracy (model specialized three times) : 8968


### Report Result

In [11]:
folder = 'adversarial_models/full_and_fine/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

In [14]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    attack_type = parts[2] + '_ff_pgd'  # untargeted
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    row = [f'{structure}']+[f'{attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model','attack_type', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


        model            attack_type   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1  untargeted-adv_ff_pgd  0.8556  0.8386  0.8760  0.8519  0.855525
1  ResNet20v2  untargeted-adv_ff_pgd  0.7979  0.7940  0.8147  0.8118  0.804600
2  ResNet56v1  untargeted-adv_ff_pgd  0.8510  0.8290  0.8410  0.8340  0.838750
3  ResNet56v2  untargeted-adv_ff_pgd  0.7930  0.8220  0.8071  0.7914  0.803375


In [15]:
folder = 'adversarial_models/naive_method/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

In [17]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_naive_untargeted.keras' or 'ResNet20v1_naive_untargeted_fine.keras'
    # Extract family
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    if parts[-1] == 'fine':
        fine = 'fine'
    else:
        fine = ''
    attack_type = f'untargeted_naive_{fine}_pgd'  # untargeted
    # no idx

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key]['acc'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc', np.nan)]
    row = [f'{structure}']+[f'{attack_type}'] + accs
    rows.append(row)

df = pd.DataFrame(rows, columns=['model','attack_type', 'acc'])

print(df)


        model                attack_type     acc
0  ResNet20v1      untargeted_naive__pgd  0.8232
1  ResNet20v1  untargeted_naive_fine_pgd  0.9051
2  ResNet20v2      untargeted_naive__pgd  0.8040
3  ResNet20v2  untargeted_naive_fine_pgd  0.9069
4  ResNet56v1      untargeted_naive__pgd  0.8513
5  ResNet56v1  untargeted_naive_fine_pgd  0.9056
6  ResNet56v2      untargeted_naive__pgd  0.8172
7  ResNet56v2  untargeted_naive_fine_pgd  0.9101


In [18]:
folder = 'adversarial_models/specialized_from_full_and_fine/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

In [20]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[3]  # ResNet20v1, etc.
    attack_type = parts[5] + '_experts'  
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    row = [f'{structure}']+[f'{attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model','attack_type', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


        model             attack_type   acc_0   acc_1   acc_2   acc_3  \
0  ResNet20v1  untargeted-adv_experts  0.9029  0.8991  0.9009  0.9034   
1  ResNet20v2  untargeted-adv_experts  0.9101  0.9060  0.9058  0.9052   
2  ResNet56v1  untargeted-adv_experts  0.9096  0.9097  0.9078  0.9123   
3  ResNet56v2  untargeted-adv_experts  0.9102  0.9128  0.9112  0.9118   

     acc_av  
0  0.901575  
1  0.906775  
2  0.909850  
3  0.911500  


In [21]:
folder = 'CIFAR10models/more_tunned/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}
print(loaded_models.keys())

dict_keys(['ResNet20v1_more_specialized_0.keras', 'ResNet20v1_more_specialized_1.keras', 'ResNet20v1_more_specialized_2.keras', 'ResNet20v1_more_specialized_3.keras', 'ResNet20v1_more_specialized_4.keras', 'ResNet20v1_more_specialized_5.keras', 'ResNet20v1_more_specialized_6.keras', 'ResNet20v1_more_specialized_7.keras', 'ResNet20v2_more_specialized_0.keras', 'ResNet20v2_more_specialized_1.keras', 'ResNet20v2_more_specialized_2.keras', 'ResNet20v2_more_specialized_3.keras', 'ResNet56v1_more_specialized_0.keras', 'ResNet56v1_more_specialized_1.keras', 'ResNet56v1_more_specialized_2.keras', 'ResNet56v1_more_specialized_3.keras', 'ResNet56v2_more_specialized_0.keras', 'ResNet56v2_more_specialized_1.keras', 'ResNet56v2_more_specialized_2.keras', 'ResNet56v2_more_specialized_3.keras'])


In [30]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_more_specialized_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[0]  # ResNet20v1, etc.
    attack_type = 'fine_tunned'  
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(len(acc_dict))]
    accs.sort(reverse = True)
    acc_av = np.nanmean(accs)
    
    row = [f'{structure}']+[f'{attack_type}'] + accs[:4] + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model','attack_type', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


        model  attack_type   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1  fine_tunned  0.9093  0.9070  0.9059  0.9052  0.904837
1  ResNet20v2  fine_tunned  0.9101  0.9095  0.9068  0.9061  0.908125
2  ResNet56v1  fine_tunned  0.9128  0.9113  0.9096  0.9069  0.910150
3  ResNet56v2  fine_tunned  0.9203  0.9174  0.9158  0.9109  0.916100


In [31]:
folder = 'CIFAR10models/ResNet/'
pattern = os.path.join(folder, '*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}
print(loaded_models.keys())

dict_keys(['n_3_v1_cifar10.keras', 'n_3_v1_cifar10_1.keras', 'n_3_v1_cifar10_2.keras', 'n_3_v1_cifar10_3.keras', 'n_3_v1_cifar10_4.keras', 'n_3_v1_cifar10_5.keras', 'n_3_v1_cifar10_6.keras', 'n_3_v1_cifar10_7.keras', 'n_3_v2_cifar10.keras', 'n_3_v2_cifar10_1.keras', 'n_3_v2_cifar10_2.keras', 'n_3_v2_cifar10_3.keras', 'n_9_v1_cifar10.keras', 'n_9_v1_cifar10_1.keras', 'n_9_v1_cifar10_2.keras', 'n_9_v1_cifar10_3.keras', 'n_9_v2_cifar10.keras', 'n_9_v2_cifar10_1.keras', 'n_9_v2_cifar10_2.keras', 'n_9_v2_cifar10_3.keras'])


In [35]:
filename = list(loaded_models.keys())[0]
base = filename.replace('.keras','')
parts = base.split('_')
structure = label_dict[int(parts[1]),int(parts[2][-1])]
structure

'ResNet20v1'

In [40]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'ResNet20v1_more_specialized_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = label_dict[int(parts[1]),int(parts[2][-1])]  # ResNet20v1, etc.
    attack_type = 'Base'  
    idx = int(parts[-1][-1])  

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(len(acc_dict))]
    accs.sort(reverse = True)
    accs = accs[:4]
    acc_av = np.nanmean(accs)
    
    row = [f'{structure}']+[f'{attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model','attack_type', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


        model attack_type   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1        Base  0.8600  0.8592  0.8533  0.8520  0.856125
1  ResNet20v2        Base  0.8208  0.8167  0.8126  0.8017  0.812950
2  ResNet56v1        Base  0.8785  0.8575  0.8486  0.8445  0.857275
3  ResNet56v2        Base  0.8355  0.8241  0.8218  0.8065  0.821975


In [20]:
import os

# Top directory and subdirectories

# Dummy test data: replace these with your actual test data
results = {}  # structure: {(structure, attack_type): {acc_0:..., acc_1:..., etc.}}

def get_models(load:bool = False, predict:bool = False):
    root_dir = 'adversarial_models'
    subdirs = ['full_and_fine', 'naive_method', 'specialized_from_full_and_fine']

    for sub in subdirs:
        path = os.path.join(root_dir, sub)
        filenames = [f for f in os.listdir(path) if f.endswith('.keras') and not f.startswith('.ipynb_checkpoints')]
        for filename in filenames:
            full_path = os.path.join(path, filename)
            if load:
                try:
                    model = keras.models.load_model(full_path)
                except Exception as e:
                    print(f"Failed to load {filename}: {e}")
                    continue

            base = filename.replace('.keras', '')
            parts = base.split('_')

            # --- Parse filename differently depending on subdirectory ---
            if sub == 'specialized_from_full_and_fine':
                # e.g., classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras
                structure = parts[3]
                attack_base = parts[5].split('-')[0]  # untargeted, targeted, cw.
                attack_type = f'specialized_{attack_base}'
                idx = int(parts[-1])
            elif sub == 'full_and_fine':
                # e.g., ResNet20v1_ff_untargeted-adv_0.keras
                structure = parts[0]
                attack_base = parts[2].split('-')[0]
                attack_type = f'ff_{attack_base}'
                idx = int(parts[-1])
            elif sub == 'naive_method':
                # e.g., ResNet56v2_naive_targeted_fine.keras or ResNet56v2_naive_targeted.keras
                structure = parts[0]
                attack_base = parts[2]
                if 'fine' in parts:
                    attack_type = f'naive_{attack_base}_tunned'
                else:
                    attack_type = f'naive_{attack_base}'
                idx = 0  # naive models don't have multiple indices
            else:
                continue

            key = (structure, attack_type)
            if key not in results:
                results[key] = {}
            results[key].update({f'acc_{idx}' : 0.})
            
            if load and predict:
                # Predict & compute accuracy
                prediction = model.predict(x_test, verbose=0)
                acc = np.sum(np.argmax(prediction, axis=-1) == y_test.reshape(-1)) / len(x_test)
                results[key][f'acc_{idx}'] = acc

    return results

In [ ]:

# --- Build dataframe ---
rows = []
for (sub, structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    model_label = f'{sub}, {structure}, {attack_type}'
    row = [model_label] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

# Display
print(df)

# Optionally save
# df.to_csv('all_models_acc.csv', index=False)

In [19]:
folder = 'adversarial_models/specialized_from_full_and_fine/'
pattern = os.path.join(folder, '*untargeted*.keras')
file_list = sorted(glob.glob(pattern))
loaded_models= {os.path.basename(f): load_model(f) for f in file_list}

dict_keys(['classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet20v1_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet20v2_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_0.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_1.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_2.keras', 'classifying_original_from_ResNet56v1_ff_untargeted-adv_3.keras', 'classifying_original_from_ResNet56v2_ff_untargeted-adv_0.keras'])


In [20]:
results = {}  # structure: {(structure, attack type): [acc_0, acc_1, acc_2, acc_3]}

for filename, model in loaded_models.items():
    # Example filename: 'classifying_original_from_ResNet20v1_ff_untargeted-adv_0.keras'
    # Extract family and index
    base = filename.replace('.keras','')
    parts = base.split('_')
    structure = parts[3]  # ResNet20v1, etc.
    attack_type = parts[5] + '_original_pgd'  # untargeted
    idx = int(parts[-1].split('-')[-1])  # last digit after -adv_

    key = (structure, attack_type)
    prediction = model.predict(x_test, verbose = 0)
    acc = np.sum(np.argmax(prediction,axis=-1) == y_test.reshape(-1)) / len(x_test)

    if key not in results:
        results[key] = {}
    results[key][f'acc_{idx}'] = acc


rows = []
for (structure, attack_type), acc_dict in results.items():
    accs = [acc_dict.get(f'acc_{i}', np.nan) for i in range(4)]
    acc_av = np.nanmean(accs)
    row = [f'{structure}, {attack_type}'] + accs + [acc_av]
    rows.append(row)

df = pd.DataFrame(rows, columns=['model', 'acc_0', 'acc_1', 'acc_2', 'acc_3', 'acc_av'])

print(df)


                            model   acc_0   acc_1   acc_2   acc_3    acc_av
0  ResNet20v1, untargeted-adv_pgd  0.9029  0.8992  0.9010  0.9033  0.901600
1  ResNet20v2, untargeted-adv_pgd  0.9101  0.9060  0.9058  0.9052  0.906775
2  ResNet56v1, untargeted-adv_pgd  0.9095  0.9097  0.9079  0.9123  0.909850
3  ResNet56v2, untargeted-adv_pgd  0.9102     NaN     NaN     NaN  0.910200
